# Board Game Behavior: An Analysis of Demographics and Gameplay Preferences

## Documentation

[Initial Project Proposal](https://github.com/sevenwwu/sevenwwu.github.io/blob/main/write-ups/initial_project_proposal_revised.md) (11/8/23)

[Milestone One](https://github.com/sevenwwu/sevenwwu.github.io/tree/main/write-ups/milestone_one_delieverable) (11/18/23)

## Data Processing and Modeling

In [ ]:
!python -m pip install pandas
!python -m pip install openpyxl
!python -m pip install seaborn
!python -m pip install scikit-learn

### Data Cleaning

In [ ]:
import pandas as pd
import numpy as np

raw_df = pd.read_excel("datasets/rawdata.xlsx",keep_default_na=False)

raw_df.info()

##### Drop Metadata

In [ ]:
dropped_df = raw_df.drop(columns=["ID","Start time","Completion time","Email","Name","Last modified time"])

dropped_df[:0]

#### Rename Raw Fields from Question to Concise Description

In [ ]:
renamed_df = dropped_df.rename(columns={
    "I am a:": "WWUStatus",
    "What is your Gender Identity?": "Gender",
    "What is your Age?\n(this field may remain blank)": "Age",
    "What is your Race?": "Race(s)",
    "What is your current Employment Status?": "EmploymentStatus",
    "Do you wear glasses or contact lenses for vision correction?": "Vision",
    "What best describes your current religious or spiritual beliefs, if any?\n": "Religiosity",
    "What is your Area of Study or Major? \n(this field may remain blank)\n": "AOS",
    "Do you enjoy playing board games, card games, or similar tabletop games?": "EnjoysBoardGames",
    "How many board/card games do you own? \n(All non-roleplay table top games are included, i.e., chess, playing cards, Uno, Magic: The Gathering, etc.)": "BoardGamesOwned",
    "About how frequently do you play board/card games?": "FrequencyOfPlay",
    "Select what best describes your engagement and style when it comes to board and card games? ": "Style",
    "I prefer games that include elements of:\n(select all that you prefer)": "PreferredElements",
    "List up to 15 board/card games that you enjoy playing. \n(abide by the format: \"Game 1\", \"Game 2\", \"Game 3\")\n(this field may remain blank)\n": "EnjoyedBoardGames",
    "On a scale from 1 to 10, how much do you enjoy playing chess, with 1 being not enjoyable at all and 10 being extremely enjoyable?": "ChessRating",
    "Select your all genres of board/card games you enjoy.\n(if your preferred genre(s) is/are not listed, please list them in the \"other\" selection)": "EnjoyedGenres"
})

renamed_df[:0]


#### Multiselect Binary/Boolean Features

##### Race(s)

In [ ]:
renamed_df["Race(s)"].value_counts()

In [ ]:
raceClean_df = renamed_df.copy()

unique_races = set()
for elements in raceClean_df["Race(s)"].dropna():
    unique_races.update(elements.split(";"))

unique_races = list(unique_races)
unique_races.sort()

In [ ]:
# Create a new column for each unique element with binary values (0 or 1 or None)
for element in unique_races:
    raceClean_df[(element + "IsRace").replace(" ", "")] = raceClean_df["Race(s)"].apply(
        lambda x: int(element in x) if pd.notna(x) else None
    )

raceClean_df = raceClean_df.drop(columns={"Race(s)","IsRace"})

##### PreferredElements

In [ ]:
raceClean_df["PreferredElements"].value_counts()

In [ ]:
preferredElementsClean_df = raceClean_df.copy()

unique_preferred_elements = set()
for elements in preferredElementsClean_df["PreferredElements"].dropna():
    unique_preferred_elements.update(elements.split(";"))
unique_preferred_elements

unique_preferred_elements = list(unique_preferred_elements)
unique_preferred_elements.sort()

In [ ]:
# Create a new column for each unique element with binary values (0 or 1 or None)
for element in unique_preferred_elements:
    preferredElementsClean_df[(element + "IsPreferredElement").replace(" ", "")] = preferredElementsClean_df["PreferredElements"].apply(
        lambda x: int(element in x) if pd.notna(x) else None
    )

preferredElementsClean_df = preferredElementsClean_df.drop(columns={"PreferredElements","IsPreferredElement"})

##### EnjoyedGenres

In [ ]:
enjoyedGenresClean_df = preferredElementsClean_df.copy()

unique_enjoyed_genres = set()
for elements in enjoyedGenresClean_df["EnjoyedGenres"].dropna():
    unique_enjoyed_genres.update(elements.split(";"))
unique_enjoyed_genres

unique_enjoyed_genres = list(unique_enjoyed_genres)
unique_enjoyed_genres.sort()

In [ ]:
# Create a new column for each unique element with binary values (0 or 1 or None)
for element in unique_enjoyed_genres:
    enjoyedGenresClean_df[(element + "IsEnjoyedGenre").replace(" ", "")] = enjoyedGenresClean_df["EnjoyedGenres"].apply(
        lambda x: int(element in x) if pd.notna(x) else None
    )

enjoyedGenresClean_df = enjoyedGenresClean_df.drop(columns={"EnjoyedGenres","IsEnjoyedGenre"})

In [ ]:
enjoyedGenresClean_df.columns.tolist()

In [ ]:
reformatColumns_df = enjoyedGenresClean_df.rename(columns=lambda x: x.replace("\xa0", ""))

reformatColumns_df.columns.tolist()

#### AOS (manual string bucketing)

In [ ]:
aos_df = reformatColumns_df
aos_df["AOS"].str.lower().unique()

In [ ]:
manual_mapping_aos = {
    "computer science": "STEM",
    "data science": "STEM",
    "eece": "STEM",
    "electrical and computer engineering": "STEM",
    "statistics": "STEM",
    "environmental studies": "STEM",
    "applied mathematics": "STEM",
    "chemistry": "STEM",
    "rml": "Other",
    "political science": "Social Studies",
    "elementary education": "Education",
    "english": "Arts & Humanities",
    "music education": "Arts & Humanities",
    "nan": "Other",
    "art": "Arts & Humanities",
    "psychology": "Social Studies",
    "psych (probably)": "Social Studies",
    "music": "Arts & Humanities",
    "environmental science - toxicology": "STEM",
    "history/museum studies": "Arts & Humanities",
    "elementary ed": "Education",
    "environmental science": "STEM",
    "mathematics": "STEM",
    "business": "Business",
    "biochem": "STEM",
    "vocal performance": "Arts & Humanities",
    "secondary education": "Education",
    "linguistics": "Arts & Humanities",
    "history": "Arts & Humanities",
    "bio/anth": "STEM",
    "marine biology": "STEM",
    "communication disorders": "Health & Medicine",
    "engineering": "STEM",
    "kinesiology": "Health & Medicine",
    "economics and mathematics": "STEM",
    "music education and german": "Arts & Humanities",
    "art p-12": "Arts & Humanities",
    "chemistry either organic or inorganic": "STEM",
    "math": "STEM",
    "electrical engineering": "STEM",
    "undecided but leaning towards engineering": "Unknown",  # Assuming lean towards STEM, but no exact match
    "medicine": "Health & Medicine",
    "rec management": "Other",  # Assuming Recreation Management
    "economics": "Social Studies",  # Close to "economics and mathematics", but economics is often considered Social Studies
    "geology": "STEM",
    "geology (paleoclimate)": "STEM",
    "visual journalism": "Arts & Humanities",
    "biology/math": "STEM",  # Combination of two STEM fields
    "behavioral neuroscience": "STEM",  # Close to "psychology" which is Social Studies, but has a heavy STEM component
    "marine biology and theater production": "STEM",  # Marine biology is STEM, theater production could be Arts, but STEM is the primary
    "anthropology": "Social Studies",
    "biology": "STEM",
    "management information systems": "Business",
    "marine bio": "STEM",
    "history/holocaust & genocide studies": "Arts & Humanities",
    "sped & eled": "Education",  # Assuming this refers to special education & elementary education
    "anthropology, communication studies": "Social Studies",
    "theatre": "Arts & Humanities",
    "studio art": "Arts & Humanities",
    "urban planning and sustainable development": "Other",  # Not a clear category, could be Social Studies or another category
    "art and design": "Arts & Humanities",
    "spanish": "Arts & Humanities",  # Language studies are often classified here
    "biochemistry": "STEM",
    "art studio": "Arts & Humanities",
    "art ed": "Arts & Humanities",
    "comm": "Other",  # Assuming "communication", but not explicitly listed, could fit Social Studies or Business,
    
    "environmental studies: eco-social justice and education emphasis": "STEM",  # Falls under Environmental Studies
    "communications": "Other",  # Often classified as Arts & Humanities
    "theatre/education": "Education",  # Falls under Education
    "undecided": "Unknown",  # Assuming still undecided as before
    "marketing": "Business",  # Falls under Business
    "communication studies": "Arts & Humanities",  # Often classified as Arts & Humanities
    "sociology": "Social Studies",  # Social Studies
    "education and public relations": "Education",  # Falls under Education
    "pre nursing": "Health & Medicine",  # Falls under Health & Medicine
    "economics/mathematics": "STEM",  # Combination of Economics and Mathematics - falls under STEM
    "mathematics secondary education": "Education",  # Falls under Education
    "dance": "Arts & Humanities",  # Often classified as Arts & Humanities
    "art studio (ba), art history": "Arts & Humanities",  # Falls under Arts & Humanities
    "narrative and folklore studies (fairhaven major)": "Arts & Humanities",  # Falls under Arts & Humanities
    "pre med and psychology": "Health & Medicine",  # Falls under Health & Medicine
    "archaeology": "Social Studies",  # Falls under Social Studies
    "neuroscience": "STEM",  # Falls under STEM
    "english literature with a teaching emphasis": "Arts & Humanities",  # Falls under Arts & Humanities
    "marine science": "STEM",  # Falls under STEM
    "fairhaven": "Other",  # Falls under Other
    "international business": "Business",  # Falls under Business
    "music composition": "Arts & Humanities",  # Falls under Arts & Humanities
    "creative writing": "Arts & Humanities",  # Falls under Arts & Humanities
    "business and sustainability": "Business",  # Falls under Business
    "early childhood education": "Education",  # Falls under Education
    "graphic design": "Arts & Humanities",  # Falls under Arts & Humanities
    "education": "Education",  # Falls under Education
    "stem": "STEM",  # Falls under STEM
    "envs": "STEM",  # Falls under STEM

    
    "behavioural neuroscience": "STEM",
    "english lit": "Arts & Humanities",
    "food equity and sustainable agriculture": "Other",  # Could be Social Studies, Business, or even STEM, unclear
    "art history and museum studies": "Arts & Humanities",
    "japanese language": "Arts & Humanities",
    "graphic design and marketing": "Arts & Humanities",  # Graphic Design is often in Arts & Humanities, Marketing is Business, but first seems primary
    "music performance major": "Arts & Humanities",
    "environment studies": "STEM",
    "business or elementary education": "Unknown",  # Could be either Business or Education 
    "marine and coastal science": "STEM",
    "undeclared, strongly thinking about history": "Unknown",  # Assuming lean towards Arts & Humanities, but no exact match
    "public health": "Health & Medicine",
    "energy policy and management": "Other",  # Could be Business, Social Studies, or STEM
    "undeclared": "Unknown",
    "fine arts": "Arts & Humanities",
    "english, history of culture": "Arts & Humanities",
    "psychology and elementary education": "Education",  # Both Psychology and Elementary Education could be Education
    "communication science and disordwrs": "Health & Medicine",
    "anthropology/history": "Social Studies",
    "special education and elementary education": "Education",
    "ibus": "Business",  # Assuming International Business
    "energy science": "STEM",
    "politics/philosophy/economics": "Social Studies",  # Combination of three Social Studies fields
    "history/social studies": "Social Studies",
    "energy": "STEM"  # Energy could be a part of STEM disciplines like Physics or Environmental Sciences
}


noncategorized_data = []

def categorize_aos_string(aos):
    if pd.isnull(aos) or aos == "n/a" or aos == "" or aos == "N/A":
        return "Unknown"  # For handling NaN values
    field_clean = aos.lower().rstrip()
    for key, category in manual_mapping_aos.items():
        if key == field_clean:
            return category
    noncategorized_data.append(field_clean)    


aos_df["AOSCat"] = aos_df["AOS"].apply(categorize_aos_string)

if (len(noncategorized_data) > 0):
    raise ValueError(f"Unknown categories: {noncategorized_data}") 

aos_df["AOSCat"].unique()

#### Shorting the Length of Single Select Answer Choices for "Style"

In [ ]:
style_df = aos_df 

style_df["Style"].unique()

In [ ]:
player_styles_mapping = {
    "Strategy-focused player, enjoying games that require planning and tactics, but also participates in more casual games.": "Strategic",
    "Situation-Specific Player, adapts style and enthusiasm based on the specific game or social context. May be casual in some situations and highly strategic in others, depending on the game being played.": "Situation-Specific",
    "Casual player, participating for fun and relaxation, without a strong focus on winning. Still willing to engage with complex games.": "Casual",
    "Simple and straightforward player, preferring uncomplicated games with easy rules.": "Simple",
    "Party-only player, primarily playing board/card games at social gatherings or parties.": "Party/Social",
    "Not interested in playing board/card games.": "Not Interested",
    "Never played or never had the opportunity to play board/card games.": "Never Played",
    "None of these describe me.": "Other",
    "Prefer not to say": "Prefer not to say"
}

def map_player_styles(string):
    for key, val in player_styles_mapping.items():
        if key in string:
            return val
    raise ValueError(f"Unknown category: {string}") 

new_style_df = style_df.copy()

new_style_df["Style"] = style_df["Style"].apply(map_player_styles)

new_style_df["Style"].unique()

#### Assigning Order to Particular Catagories

In [ ]:
cat_order_df = new_style_df.copy()

cat_order_df["BoardGamesOwned"] = cat_order_df["BoardGamesOwned"].apply(lambda x: "0" if x == "I do not own any board/card games.\xa0" else x)

cat_order_df["BoardGamesOwned"] = pd.Categorical(cat_order_df["BoardGamesOwned"],
    categories=["Prefer not to say","0", "1 or 2", "2 to 5", "5 to 10", "10 to 20", "More than 20"],ordered=True)


cat_order_df["FrequencyOfPlay"] = pd.Categorical(cat_order_df["FrequencyOfPlay"],
    categories=["Daily","Several times a week","Weekly","Several times a month","Monthly","Every few months","Rarely/Seldom","Never","Prefer not to say",],ordered=True)


cat_order_df["ChessRating"] = cat_order_df["ChessRating"].apply(lambda x: "Unknown" if x == "" else x)

cat_order_df["ChessRating"] = pd.Categorical(cat_order_df["ChessRating"].astype(str),
    categories=["Unknown","1","2","3","4","5","6","7","8","9","10"], ordered=True)


#### Convert Features to Int

In [ ]:
convert_df = cat_order_df.copy()
convert_df["Age"] = convert_df["Age"].apply(lambda x: -1 if x == "" else x)
convert_df["Age"] = convert_df["Age"].astype(int)

In [ ]:
cleaned_df = convert_df

cleaned_df.to_csv("datasets/cleaned.csv")

### Exploratory Data Analysis

In [ ]:
single_select_features = ['WWUStatus', 'Gender', 'Age', 'EmploymentStatus', 'Vision',
       'Religiosity', 'AOSCat', 'EnjoysBoardGames', 'BoardGamesOwned',
       'FrequencyOfPlay', 'Style', 'ChessRating']

race_bool_features = ['WhiteIsRace','AsianIsRace', 'BlackorAfricanAmericanIsRace', 'FilipinoIsRace',
       'HispanicorLatinoIsRace','MixedethnicityIsRace', 'MultiracialIsRace', 'NativeAmericanorAmericanIndianIsRace',
       'NativeHawaiianorPacificIslanderIsRace', 'PrefernottosayIsRace',]
       
element_bool_features = ['Conflict/CompetitionIsPreferredElement',
       'CooperationIsPreferredElement',
       'Heavy/ImmersiveThemingIsPreferredElement', 'LuckIsPreferredElement',
       'Party/Low-StakesIsPreferredElement',
       'Puzzle-SolvingIsPreferredElement',
       'SocialDeduction/HiddenRoleIsPreferredElement',
       'StrategyIsPreferredElement', 'TriviaIsPreferredElement',]

genre_bool_features = ['AbstractStrategyIsEnjoyedGenre', 'AdventureIsEnjoyedGenre',
       'AnimalsIsEnjoyedGenre', 'AuctionIsEnjoyedGenre', 'CardIsEnjoyedGenre',
       'CardDraftingIsEnjoyedGenre', 'CivilizationIsEnjoyedGenre',
       'CooperativeIsEnjoyedGenre', 'Deck-BuildingIsEnjoyedGenre',
       'DeductionIsEnjoyedGenre', 'EconomicIsEnjoyedGenre',
       'EducationalIsEnjoyedGenre', 'ExplorationIsEnjoyedGenre',
       'FantasyIsEnjoyedGenre', 'FarmingIsEnjoyedGenre',
       'FightingIsEnjoyedGenre', 'HorrorIsEnjoyedGenre', 'LuckIsEnjoyedGenre',
       'MedievalIsEnjoyedGenre', 'MemoryIsEnjoyedGenre',
       'MiniaturesIsEnjoyedGenre', 'PartyIsEnjoyedGenre',
       'PiratesIsEnjoyedGenre', 'PoliticalIsEnjoyedGenre',
       'PuzzleIsEnjoyedGenre', 'RacingIsEnjoyedGenre',
       'Role-PlayingIsEnjoyedGenre', 'RollandMoveIsEnjoyedGenre',
       'ScienceFictionIsEnjoyedGenre',
       'SocialDeduction/HiddenRoleIsEnjoyedGenre', 'SportsIsEnjoyedGenre',
       'StrategyIsEnjoyedGenre', 'TerritoryBuildingIsEnjoyedGenre',
       'Tile-LayingIsEnjoyedGenre', 'TrainsIsEnjoyedGenre',
       'TransportationIsEnjoyedGenre', 'TravelIsEnjoyedGenre',
       'TriviaIsEnjoyedGenre', 'WarIsEnjoyedGenre', 'WordIsEnjoyedGenre',
       'WorkerPlacementIsEnjoyedGenre', 'WorldWarIIIsEnjoyedGenre',
       'ZombiesIsEnjoyedGenre',]

free_form_features = ['AOS','EnjoyedBoardGames']


In [ ]:
len(cleaned_df.columns)

In [ ]:
len(single_select_features + race_bool_features + element_bool_features + genre_bool_features + free_form_features)

In [ ]:
cleaned_df["Religiosity"].value_counts()

In [ ]:
religious_bucketing = {
    "Christianity": "Religious",
    "Judaism": "Religious",
    "Islam": "Religious",
    "Hinduism": "Religious",
    "Buddhism": "Religious",
    "Lutheran": "Religious",
    "Pagan": "Religious",
    "Paganism": "Religious",
    "Unitarian": "Religious",
    "I believe a god exists but don’t follow any religious texts": "Religious",
    "Toaism": "Religious",
    "pagan": "Religious",

    "Atheism": "Not Religious",
    "No specific belief": "Not Religious",
    "Agnosticism": "Not Religious",
    "Spiritual, not affiliated with a specific religion": "Not Religious",
    "Science": "Not Religious",
    "being with oneself in connection to everything": "Not Religious",
    "Prefer not to say": "Not Religious",
}

#### Defining Functions for Analyzing Categorical Data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import pandas as pd



def categories_against_category(df, categories, category):
    for feature in categories:
        print(df.groupby(category)[feature].value_counts())
        # Compute percentage for each category within each cluster
        df_percent = df.groupby(category)[feature].value_counts(normalize=True).rename('Percentage').reset_index()
        df_percent["Percentage"] *= 100  # Convert to %

        # Create seaborn barplot
        plt.figure(figsize=(8, 6))
        sns.barplot(x=feature, y='Percentage', hue=category, data=df_percent)
        plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
        plt.xticks(rotation=90)
        plt.show()

def bools_against_category(df,bools,category):
    # Initialize an empty dataframe to store your percentages
    percentages = pd.DataFrame()

    # Loop through the bool_columns 
    for col in bools:
        # Compute the percentages of True (=1) occurrences in each cluster
        percents = df.groupby(category,observed=True)[col].mean().mul(100).reset_index()
        percents['Feature'] = col
        percents.rename({col: 'Percentage'}, axis=1, inplace=True)

        # Append the computed percentages to your dataframe
        percentages = pd.concat([percentages, percents])

    # Once your dataframe is ready, you can plot it using seaborn
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Feature', y='Percentage', hue=category, data=percentages)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
    plt.xticks(rotation=90)
    plt.title(f"Percentage of 'True' Occurrences in Each Feature by {category}")
    plt.show()

# Intended for single-select/categorical feature comparison
# `x` should be a Panda Series
# `y` should be a Panda Series
def plot_count_and_percentage_heatmaps(x, y):
    data = pd.DataFrame({x.name: x, y.name: y})

    count_matrix = pd.crosstab(data[x.name], data[y.name])
    
    percent_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0) * 100

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Heatmap for counts
    sns.heatmap(count_matrix, annot=True, fmt="d", cmap="YlGnBu", cbar=True, ax=ax[0])
    ax[0].set_title("Count Matrix of Categories")
    ax[0].set_xlabel(y.name)
    ax[0].set_ylabel(x.name)

    # Heatmap for percentages
    sns.heatmap(percent_matrix, annot=True, fmt=".2f", cmap="YlGnBu", cbar=True, ax=ax[1],vmin=0.0, vmax=100.0)
    ax[1].set_title("Percentage Matrix of Categories")
    ax[1].set_xlabel(y.name)
    ax[1].set_ylabel(x.name)

    plt.tight_layout()
    plt.show()

# Intended for categorical feature comparison against several bool fields
# `data` should be a Pandas Data Frame
# `x` should be a string that refers to the categorical feature in `data`
# `y` should be an array of strings that refer to the several bool features in `data`
def plot_heatmap_of_bool_features_percent(data, x, y):
    df_melted = data.melt(id_vars=x, value_vars=y, var_name="Bool", value_name="True")

    df_pivot = df_melted.groupby([x,"Bool"]).sum().reset_index().pivot(index=x, columns="Bool", values="True")

    total = data[x].value_counts()

    df_pivot = df_pivot.join(total)
    

    df_pivot.loc[:, df_pivot.columns != "count"] = ((df_pivot.loc[:, df_pivot.columns != "count"].div(df_pivot["count"], axis=0)) * 100).round(4)
    
    print(df_pivot["count"])
    df_pivot = df_pivot.drop(columns=["count"])
    
    df_pivot = df_pivot.transpose()

    sns.heatmap(df_pivot, annot=True, cmap="Blues", fmt=".2f",vmin=0.0, vmax=100.0)

# Intended for categorical feature comparison against several bool fields
# `data` should be a Pandas Data Frame
# `x` should be a string that refers to the categorical feature in `data`
# `y` should be an array of strings that refer to the several bool features in `data`
def plot_heatmap_of_bool_features_val_count(data, x, y):
    df_melted = data.melt(id_vars=x, value_vars=y, var_name="Bool", value_name="True")

    df_pivot = df_melted.groupby([x,"Bool"]).sum().reset_index().pivot(index=x, columns="Bool", values="True")

    total = data[x].value_counts()

    df_pivot = df_pivot.join(total)
    
    df_pivot.rename(columns={"count": "UniqueMembersOfParticular" + x}, inplace=True)
    
    df_pivot = df_pivot.transpose()
    # Plot
    sns.heatmap(df_pivot, annot=True, cmap="Blues", fmt=".2f")

# Used to reduce the number of categories in a feature by setting the value to "Other" for insufficient response quantity of specific catagories
# `df` should be a Pandas Data Frame
# `col_name` should be the string name of the column/feature you'd like to filter
# `threshold` should be the minimum count needed to maintain the categorical variable 

# For example, if `Age` gets a bunch of responses between 18 and 24, and you get a few 40s and one 60. You can use this function to reduce the 
# different categorical variables to simply 18 through 24 and "Other"
def filter_threshold(df, col_name, threshold):
    df_copy = df.copy() 
    counts = df_copy[col_name].value_counts()
    df_copy.loc[df_copy[col_name].isin(counts[counts < threshold].index), col_name] = "Other"
    return df_copy

#### Distribution of Features

The following code is used to filter out feature responses that are underrepresented or minimal in survey data, thus preventing inaccurate conclusions that could arise from only looking at a small sample of a population. Most of features graphed below have at least two prominent catagories for which to perform data analysis, which is sufficient. This includes:

- Gender
- Age (within the range of 18 to 24)
- Employment Status (at least for Part-Time and Unemployed)
- Vision
- Area of Study Catagories
- Number of Board Games Owned
- Frequency of Play
- Style
- Chess Rating

However, a few fields did not get sufficient diverse responses to conclude anything, this includes:
- WWU Status
  - Nearly everyone who took this survey was a student
- Religiosity
  - Most people who filled out this survey were secular, and the largest minority religious group is simply too small to make any meaningful conclusions
- Does person enjoys board games
  - The overwhelming majority of people who filled out this survey liked playing board games making relationships between demographics hard to draw

In [ ]:
for feature in single_select_features:
    sns.countplot(x=feature, data=cleaned_df)
    plt.xticks(rotation=90) 
    plt.show()

#### Distribution of Features (cont'd)

The following questions were multi-select and resulted in bool fields. Because of this, we need to display the data slightly differently: totalling all "True"/selected values from the related features.

The multi-select questions were in regard to Race, Preferred Gameplay Elements, and Enjoyed Genres.

Preferred Gameplay Elements and Enjoyed Genres got a sufficient distribution of responses, however, Race did not receive sufficiently diverse responses for analysis.

In [ ]:
true_counts = {}

for col in race_bool_features:
    true_counts[col] = cleaned_df[cleaned_df[col] == 1].shape[0]

true_counts_series = pd.Series(true_counts)

sns.barplot(true_counts_series)
plt.xticks(rotation=90)
plt.show()

In [ ]:
true_counts = {}

for col in element_bool_features:
    true_counts[col] = cleaned_df[cleaned_df[col] == 1].shape[0]

true_counts_series = pd.Series(true_counts)

sns.barplot(true_counts_series)
plt.xticks(rotation=90)
plt.show()

In [ ]:
true_counts = {}

for col in genre_bool_features:
    true_counts[col] = cleaned_df[cleaned_df[col] == 1].shape[0]

true_counts_series = pd.Series(true_counts)

sns.barplot(true_counts_series)
plt.xticks(rotation=90)
plt.show()

#### (Random) Exploratory Analysis

This section is dedicated to graphing random sufficient features against one another in hopes of uncovering hidden relationships.

In [ ]:
# There were many different responses for various types of non-binary gender identities, because of this, graphing the data becomes a bit muddled
# This is solved by grouping the non-binary responses into an "Other" category
genderForAnalysis_df = filter_threshold(cleaned_df,"Gender",25)
genderForAnalysis_df["Gender"].value_counts()

In [ ]:
genderForAnalysis_df = genderForAnalysis_df[genderForAnalysis_df["Gender"] != "Other"]

In [ ]:
categories_against_category(genderForAnalysis_df,single_select_features,"Gender")

In [ ]:
bools_against_category(genderForAnalysis_df,race_bool_features,"Gender")
bools_against_category(genderForAnalysis_df,element_bool_features,"Gender")
bools_against_category(genderForAnalysis_df,genre_bool_features,"Gender")

`All Single Select` Against `ChessRating` Box Plot

There does appear to be a substantial difference in ratings between Men and Women within in the data.

In [ ]:
genderForAnalysis_df["Gender"].value_counts()

In [ ]:
modified_df = cleaned_df.copy()
modified_df = filter_threshold(modified_df,"Gender",30)
modified_df = filter_threshold(modified_df,"Age",10)
modified_df = filter_threshold(modified_df,"EmploymentStatus",30)
modified_df["Religiosity"] = modified_df["Religiosity"].value_counts()
modified_df = filter_threshold(modified_df,"AOSCat",30)


features = single_select_features.copy()
features.remove("WWUStatus")
for feature in features:
    print(modified_df[feature].value_counts())
    plt.figure(figsize=(20, 10))
    sns.boxenplot(x=modified_df[feature],y=modified_df["ChessRating"])
    plt.ylim(reversed(plt.ylim()))
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Concatenate the boolean columns with the "ChessRating" column
df_bool = cleaned_df[element_bool_features + ['ChessRating']]

# Reshape your DataFrame so that each boolean feature and its corresponding "ChessRating" are in a single row
df_melt = df_bool.melt(id_vars='ChessRating', var_name='Feature', value_name='Value')

# Select only the rows where the category is marked as True
df_melt = df_melt[df_melt['Value'] == 1]

# Now you can plot everything on the same plot
plt.figure(figsize=(20, 10))
sns.boxenplot(x='Feature', y='ChessRating', data=df_melt)
plt.ylim(reversed(plt.ylim()))
plt.xticks(rotation=90)
plt.title('Distribution of ChessRating for Each Feature')
plt.show()

In [ ]:
# Concatenate the boolean columns with the "ChessRating" column
df_bool = cleaned_df[genre_bool_features + ['ChessRating']]

# Reshape your DataFrame so that each boolean feature and its corresponding "ChessRating" are in a single row
df_melt = df_bool.melt(id_vars='ChessRating', var_name='Feature', value_name='Value')

# Select only the rows where the category is marked as True
df_melt = df_melt[df_melt['Value'] == 1]

# Now you can plot everything on the same plot
plt.figure(figsize=(20, 10))
sns.boxenplot(x='Feature', y='ChessRating', data=df_melt)
plt.ylim(reversed(plt.ylim()))
plt.xticks(rotation=90)
plt.title('Distribution of ChessRating for Each Feature')
plt.show()

In [ ]:
modified_df = cleaned_df.copy()
modified_df = filter_threshold(modified_df,"Gender",30)
modified_df = filter_threshold(modified_df,"Age",10)
modified_df = filter_threshold(modified_df,"EmploymentStatus",30)
modified_df["Religiosity"] = modified_df["Religiosity"].map(religious_bucketing)
modified_df = filter_threshold(modified_df,"AOSCat",30)

features = single_select_features.copy()
features.remove("WWUStatus")
for feature in features:
    print(modified_df[feature].value_counts())
    bools_against_category(modified_df,element_bool_features,feature)

In [ ]:
modified_df = cleaned_df.copy()
modified_df = filter_threshold(modified_df,"Gender",30)
modified_df = filter_threshold(modified_df,"Age",10)
modified_df = filter_threshold(modified_df,"EmploymentStatus",30)
modified_df["Religiosity"] = modified_df["Religiosity"].map(religious_bucketing)
modified_df = filter_threshold(modified_df,"AOSCat",30)

features = single_select_features.copy()
features.remove("WWUStatus")
for feature in features:
    print(modified_df[feature].value_counts())
    bools_against_category(modified_df,genre_bool_features,feature)

In [ ]:
selected_rows = cleaned_df.copy()
selected_rows["FrequencyOfPlay"] = cleaned_df["FrequencyOfPlay"].apply(lambda x: "At Least Weekly" if x in ["Daily","Several times a week","Weekly"] else x)

selected_rows["FrequencyOfPlay"] = pd.Categorical(selected_rows["FrequencyOfPlay"],
    categories=["At Least Weekly","Several times a month","Monthly","Every few months","Rarely/Seldom","Never","Prefer not to say"],ordered=True)

bools_against_category(selected_rows,element_bool_features,"BoardGamesOwned")
bools_against_category(selected_rows,genre_bool_features,"BoardGamesOwned")

In [ ]:
modified_df = cleaned_df.copy()
modified_df = filter_threshold(modified_df,"Style",20)
modified_df = modified_df[modified_df["Style"] != "Other"]


print(modified_df["Style"].value_counts())
bools_against_category(modified_df,element_bool_features,"Style")
bools_against_category(modified_df,genre_bool_features,"Style")

In [ ]:
modified_df = cleaned_df.copy()
modified_df = filter_threshold(modified_df,"AOSCat",20)

print(modified_df["AOSCat"].value_counts())

features = single_select_features.copy()
features.remove("WWUStatus")
categories_against_category(modified_df,features,"AOSCat")

In [ ]:
bools_against_category(modified_df,element_bool_features,"AOSCat")
bools_against_category(modified_df,genre_bool_features,"AOSCat")

#### Machine Learning Clustering for Exploratory Analysis

In [66]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.calibration import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier


from sklearn import metrics
from sklearn.cluster import KMeans

data = cleaned_df.copy()
X = data.drop(columns=["EnjoyedBoardGames","AOS"])

onehot_and_ordinal_transform = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["WWUStatus","EmploymentStatus","Vision","Religiosity","EnjoysBoardGames","Gender","Style","AOSCat"]),
        ("ordinal", make_pipeline(OrdinalEncoder(categories=[["Prefer not to say", "0", "1 or 2", "2 to 5", "5 to 10", "10 to 20", "More than 20"],
                                               ["Daily", "Several times a week", "Weekly", "Several times a month", "Monthly", "Every few months", "Rarely/Seldom", "Never", "Prefer not to say"],
                                               ["Unknown", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]]),StandardScaler()),
                                                
                                               ["BoardGamesOwned",
                                                "FrequencyOfPlay",
                                                "ChessRating"]),
        ("scale numeric types", StandardScaler(), ["Age"])                      
    ], remainder="passthrough"
    )


X_transformed = onehot_and_ordinal_transform.fit_transform(X)



for i in range(2,10):
    k_means_model = KMeans(n_clusters=i, random_state=5, n_init=10)
    clusters = k_means_model.fit_predict(X_transformed)
    score = metrics.silhouette_score(X_transformed, clusters)
    print(f'Fitting for {i} clusters')
    print(f'score: {score}')
    print()


Fitting for 2 clusters
score: 0.11863466543320195

Fitting for 3 clusters
score: 0.05927103971485946

Fitting for 4 clusters
score: 0.053179399224910384

Fitting for 5 clusters
score: 0.05032617609897155

Fitting for 6 clusters
score: 0.05323918691389219

Fitting for 7 clusters
score: 0.05310292323114009

Fitting for 8 clusters
score: 0.04638618458724722

Fitting for 9 clusters
score: 0.047447314603493905



In [67]:
k_means_model = KMeans(n_clusters=2, random_state=5, n_init=10)
clusters = k_means_model.fit_predict(X_transformed)
score = metrics.silhouette_score(X_transformed, clusters)
print(f'Fitting for {2} clusters')
print(f'score: {score}')
print()

Fitting for 2 clusters
score: 0.11863466543320195



In [ ]:
X["Cluster"] = k_means_model.fit_predict(X_transformed)
X

In [ ]:
X["Cluster"].value_counts()

In [ ]:
X["Cluster"].value_counts(normalize=True)

In [ ]:
categories_against_category(X,single_select_features,"Cluster")

In [ ]:
bools_against_category(X,race_bool_features,"Cluster")
bools_against_category(X,element_bool_features,"Cluster")
bools_against_category(X,genre_bool_features,"Cluster")

In [ ]:
from sklearn.cluster import SpectralClustering

data = cleaned_df.copy()
X = data.drop(columns=["EnjoyedBoardGames","AOS"])

onehot_and_ordinal_transform = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["WWUStatus","EmploymentStatus","Vision","Religiosity","EnjoysBoardGames","Gender","Style","AOSCat"]),
        ("ordinal", make_pipeline(OrdinalEncoder(categories=[["Prefer not to say", "0", "1 or 2", "2 to 5", "5 to 10", "10 to 20", "More than 20"],
                                               ["Daily", "Several times a week", "Weekly", "Several times a month", "Monthly", "Every few months", "Rarely/Seldom", "Never", "Prefer not to say"],
                                               ["Unknown", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]]),StandardScaler()),
                                                
                                               ["BoardGamesOwned",
                                                "FrequencyOfPlay",
                                                "ChessRating"]),
        ("scale numeric types", StandardScaler(), ["Age"])                      
    ], remainder="passthrough"
    )


X_transformed = onehot_and_ordinal_transform.fit_transform(X)



for i in range(2,10):
    k_means_model = SpectralClustering(n_clusters=i, random_state=5, n_init=10)
    clusters = k_means_model.fit_predict(X_transformed)
    score = metrics.silhouette_score(X_transformed, clusters)
    print(f'Fitting for {i} clusters')
    print(f'score: {score}')
    print()


### Machine Learning Predictive Model

#### First Attempt at Building a Machine Learning Model, Predict Gender from Board Game Preferences

In [ ]:
# load dataset
data = cleaned_df.copy()
data = data[["Gender","BoardGamesOwned","FrequencyOfPlay","Style","ChessRating",
     
     "Party/Low-StakesIsPreferredElement","SocialDeduction/HiddenRoleIsPreferredElement","TriviaIsPreferredElement",

     "AnimalsIsEnjoyedGenre","CardIsEnjoyedGenre","FantasyIsEnjoyedGenre","HorrorIsEnjoyedGenre","LuckIsEnjoyedGenre",
     "PuzzleIsEnjoyedGenre","Role-PlayingIsEnjoyedGenre","RollandMoveIsEnjoyedGenre","ScienceFictionIsEnjoyedGenre",
     "SocialDeduction/HiddenRoleIsEnjoyedGenre","SportsIsEnjoyedGenre","TriviaIsEnjoyedGenre","WarIsEnjoyedGenre",
     "WorldWarIIIsEnjoyedGenre","ZombiesIsEnjoyedGenre"]]

# data = data.drop(columns=["WWUStatus","EmploymentStatus","Vision","Religiosity","AOS","EnjoysBoardGames",
#                           "EnjoyedBoardGames"])



data = filter_threshold(data,"Gender",20)
data = data[data["Gender"] != "Other"]
# data = filter_threshold(data,"AOSCat",15)
data = filter_threshold(data,"Style",15)


In [ ]:
X, y = data.drop(columns=["Gender"]), data["Gender"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, test_size=0.5, stratify=y, random_state=40)


onehot_and_ordinal_transform = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["Style"]),
        ("ordinal", make_pipeline(OrdinalEncoder(categories=[["Prefer not to say", "0", "1 or 2", "2 to 5", "5 to 10", "10 to 20", "More than 20"],
                                               ["Daily", "Several times a week", "Weekly", "Several times a month", "Monthly", "Every few months", "Rarely/Seldom", "Never", "Prefer not to say"],
                                               ["Unknown", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]]),StandardScaler()),
                                                
                                               ["BoardGamesOwned",
                                                "FrequencyOfPlay",
                                                "ChessRating"])                         
    ], remainder="passthrough"
)


logistic_pipe = make_pipeline(
    onehot_and_ordinal_transform,
    LogisticRegression(max_iter=10000)
)

k_neighbors_classifier_pipe = make_pipeline(
    onehot_and_ordinal_transform,
    KNeighborsClassifier()
)

decision_tree_classifier_pipe = make_pipeline(
    onehot_and_ordinal_transform,
    DecisionTreeClassifier()
)

random_forest_classifier_pipe = make_pipeline(
    onehot_and_ordinal_transform,
    RandomForestClassifier(random_state=40)
)

dummy_classifier = DummyClassifier(strategy='most_frequent')


current_pipe = logistic_pipe

print(f"Model Cross Val Scores: {cross_val_score(current_pipe,X_train,y_train,cv=4,scoring=make_scorer(accuracy_score))}")
print(f"Dummy Cross Val Scores: {cross_val_score(dummy_classifier,X_train,y_train,cv=4,scoring=make_scorer(accuracy_score))}")

In [ ]:
# Should be used when certain model performs well

current_pipe.fit(X_train,y_train)
conf_matrix = confusion_matrix(y_test, current_pipe.predict(X_test),labels=["Man","Woman","Other"])

print(conf_matrix)

print(f"Model Accuracy Score: {accuracy_score(current_pipe.predict(X_test),y_test)}")



dummy_classifier.fit(X_train, y_train)
y_pred = dummy_classifier.predict(X_test)

print(f"Dummy Accuracy Score: {accuracy_score(y_test, y_pred)}")

#### Second Attempt at Machine Learning Model, Predicting Different Board Game Stats Based on Demographics

In [ ]:
# load dataset
data = cleaned_df.copy()
data = data[['Gender', 'Age', 
            'AOSCat',
       
            "BoardGamesOwned","FrequencyOfPlay","Style","ChessRating",]]

# data = data.drop(columns=["WWUStatus","EmploymentStatus","Vision","Religiosity","AOS","EnjoysBoardGames",
#                           "EnjoyedBoardGames"])



data = filter_threshold(data,"Gender",20)
# data = data[data["Gender"] != "Other"]
#filter_threshold(data,"Age",10)["Age"].value_counts()
# data = filter_threshold
data = filter_threshold(data,"AOSCat",15)
data = filter_threshold(data,"Style",15)

data["FrequencyOfPlay"] = data["FrequencyOfPlay"].apply(lambda x: "At Least Weekly" if x in ["Daily","Several times a week","Weekly"] else x)


data["FrequencyOfPlay"] = pd.Categorical(data["FrequencyOfPlay"],
    categories=["At Least Weekly","Several times a month","Monthly","Every few months","Rarely/Seldom","Never","Prefer not to say"],ordered=True)

data["ChessRating"] = data["ChessRating"].replace('Unknown', np.nan).astype(float)

In [ ]:
data = data.dropna(subset=['ChessRating'])

In [ ]:
len(data)

In [ ]:
X, y = data.drop(columns=["BoardGamesOwned","FrequencyOfPlay","Style","ChessRating"]), data["ChessRating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, test_size=0.5, stratify=y, random_state=40)


onehot_and_ordinal_transform = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"), ["Gender","AOSCat", ]),
        # ("ordinal", make_pipeline(OrdinalEncoder(categories=[["Prefer not to say", "0", "1 or 2", "2 to 5", "5 to 10", "10 to 20", "More than 20"],
        #                                        ["Daily", "Several times a week", "Weekly", "Several times a month", "Monthly", "Every few months", "Rarely/Seldom", "Never", "Prefer not to say"],
        #                                        ["Unknown", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]]),StandardScaler()),
                                                
        #                                        ["BoardGamesOwned",
        #                                         "FrequencyOfPlay",
        #                                         "ChessRating"])                         
    ], remainder="passthrough"
)


logistic_pipe = make_pipeline(
    onehot_and_ordinal_transform,
    LogisticRegression(max_iter=10000)
)


dummy_classifier = DummyClassifier(strategy='uniform')


current_pipe = logistic_pipe

print(f"Model Cross Val Scores: {cross_val_score(current_pipe,X_train,y_train,cv=4)}")
print(f"Dummy Cross Val Scores: {cross_val_score(dummy_classifier,X_train,y_train,cv=4)}")

In [ ]:
current_pipe.fit(X_train,y_train)

current_pipe.score(X_train,y_train)

In [ ]:
current_pipe.fit(X_train,y_train)
conf_matrix = confusion_matrix(y_test, current_pipe.predict(X_test))

print(conf_matrix)

print(f"Model Accuracy Score: {accuracy_score(current_pipe.predict(X_test),y_test)}")



dummy_classifier.fit(X_train, y_train)
y_pred = dummy_classifier.predict(X_test)

print(f"Dummy Accuracy Score: {accuracy_score(y_test, y_pred)}")

In [ ]:
single_select_features